# Modelo de Detección de la Dislexia aplicando Machine Learning

Se va a desarrollar un modelo en Machine Learning que sea capaz de detectar si, tras contestar a 32 questiones y teniendo en cuenta el género, la lengua y la edad, una persona es disléxica o no. Cada modelo se va a entrenar y testear con dos datasets distintos: desktop.csv (Train) y tablet.csv (Test).


Para ello se va a emplear Machine Learning, importando las librerías de python. Los algoritmos que se van a implementar son:
1. K-Nearest Neighbors
2. Logistic Regression
3. Support Vector Machines
4. Random Forest
5. Tree Decision

Y finalmente, en caso de que de tiempo:

6. Neural Networks 

A contiuación, se van a importar las librerías necesarias para desarrollar el modelo de Machine Learning:
- Pandas: proporcina herramientas para el análisis de datos. Mediante esta herramienta se va leer el documento .csv donde se encuentran los datos y se van a manipular con el propósito de diseñar el modelo de Mchine Learning.
- NumPy y Matplotlib: se emplearán para el análisis y la visualización de datos. NumPy permite realizar operación y  matemáticas y manejar datos numéricos de manera eficiente y efectiva. Por otro lado, Matplotlib permite crear gráficas para visualizar los datos de manera simple y clara. 
- Seaborn: al igual que Matplotlib, se trata de una librería de visualización de datos. Sin embargo, ofrece una visualización estadística, la cua va a ser de gran utilidad para analizar la distribución de los datos.
- Warnings: gestionará las advertencias que surjan durante la ejecución del programa. En concreto se va a hacer uso de `warnings.filterwarnings('ignore')` consiguiendo ignorar las posibles advertencias.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
%matplotlib inline 
warnings.filterwarnings('ignore')

A su vez, para el entrenamiento y el testeo de los distintos algoritmos de Machine Learning, se van a importar las librerías **Scikit-Learn** y para el manejo de desproporción entre casos positivos y negativos se va a importar **Imbalanced-Learn**.

In [ ]:
# ! pip install scikit-learn
# ! pip install imblearn
# ! pip install seaborn

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.model_selection import KFold
import seaborn as sns

## Modeling Preparation

Se va a comenzar leyendo los datos y exportándolos a la variable *data*, la cual va a contener la información con la que se va a construir el modelo. Mediante la librería **pandas** se va a acceder a la información, siendo capaz de:
- Comprobar si existen valores nulos
- Transformar todos los datos a valores numéricos
- Gestionar los valores nulos
- Analizar si hay variedad dentro de cada dataset

La información que se va a obtener es:

1. Genero (hombre o mujer)
2. Lengua nativa es el español
3. Lengua nativa distinta al español
4. Edad 
5. Información relacionada con las preguntas
6. Disléxico: sí o no

La información relacionada con las preguntas contiene el número de clicks que se realizan en cada ejercicio (*Clicks*), diferenciando en respuestas correctas (*Hits*) e incorrectas (*Misses*). A su vez, se cuenta con el resultado final (*Score*), que se obtiene a partir de la cantidad de aciertos por cuestión, junto con la precisión de la respuesta (*Accuracy = Hits/Clicks*) y el ratio de fallo (*Missrate = Misses/Clicks*).

Mediante estos valores se pretende predecir si una persona tiene dislexia o no.

In [ ]:
data = pd.read_csv("../data/Dyt-desktop.csv", delimiter=";")
pd.set_option("display.max_columns" , None)

In [ ]:
# first look train dataset
data.head()
data.iloc[:, :10].dtypes
data.shape

Se ha creado una función `prep_Data(df)`que toma como atributo de entrada un dataset y transforma los valores de las columnas *Gender, Nativelang, Otherlang y Dyslexia* en 0's o 1's, en función de su valor incial. De esta forma solo se trabajará con valores numéricos.

In [ ]:
def prep_Data(ds) :
    #ds.column -> access to the column specified
    #map() -> itarates through the column specified
    ds['Gender']=ds.Gender.map({'Male': 0, 'Female': 1})
    ds['Dyslexia']=ds.Dyslexia.map({'No': 0, 'Yes': 1})
    ds['Nativelang']=ds.Nativelang.map({'No': 0, 'Yes': 1})
    ds['Otherlang']=ds.Otherlang.map({'No': 0, 'Yes': 1})

In [ ]:
prep_Data(data)

Al contar con una desproporción muy elevada entre casos positivos y negativos de dislexia entre los datos se va a optar por aumentar el número de datos del dataset, mediante la librería **RandomOverSampler**, de forma que la cantidad de sies sea igual que la de noes.

In [ ]:
# Class count
count_dyslexia_no, count_dyslexia_yes = data.Dyslexia.value_counts()

# Divide by class
dyslexia_no_df = data[data['Dyslexia'] == 0]
dyslexia_yes_df = data[data['Dyslexia'] == 1]

# Oversample 0-class and concat the DataFrames of both class
dyslexia_yes_df = dyslexia_yes_df.sample(count_dyslexia_no, replace = True)
data = pd.concat([dyslexia_no_df, dyslexia_yes_df], axis=0)

print('Random under-sampling:')
print(data.Dyslexia.value_counts())

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
print("Duplicated data from  Data Set: ",data.index.duplicated().sum())

In [ ]:
data.head()

Tras transformar todos los valores del dataframe en numéricos, se va a asegurar que todos los valores sean tipo flotantes mediante la instrucción: `dataset.apply(lambda x: x.astype('float', errors='ignore'))` donde se aplica el cambio de *int* a *flot* mediante la función `astype()` dentro de una función *lambda* que recorre cada columna, accediendo a su contenido y modificándolo. 

In [ ]:
data = data.apply(lambda x: x.astype('float', errors='ignore'))

Una vez se haya transformado el tipo de valor con el que se está trabajando se va a comprobar que solo contiene valores del tipo *float64*:

In [ ]:
#check the data type of each column
for col in data.columns:
    if data[col].dtypes != "float64":
        print("Column {} is not a float, it is {}".format(col, data[col].dtypes))

Una vez exportados los datos se va a comporbar si hay valores nulos mediante la función `[col for col in test_df.columns if train_df[col].isnull().any()]`. 
- Mediante el bucle `for col in` se van a recorrer todas las columna y acceder a sus valores.  
- Mediante el condicional `if train_df[col].isnull().any()`se va a comprobar si alguno de los datos de la columna que se está analizando es nulo o no. 

En caso de que se encuentre un valor nulo, el atributo de salida `col` tomará el valor de dicha columna y se añadirá a una lista (función entre `[ ]`).

In [ ]:
data.describe(include = [np.number])
data.isnull().values.any()
data_null_values = data.isna().sum().sum()
data_null_col = [col for col in data.columns if data[col].isnull().any()]
data_null_col

### Handling outliers

Se va a prescindir de todos los valores mayores a uno en las columnas *Accuracy* y *Missrate*. Los valores mencionados representan porcentajes, por lo que no pueden superar la unidad, sin embargo, si se estudia en detalle cada variable existen valores que no cumplen este requisito dando lugar a error si no se corrige.

In [ ]:
def remove_outliers_manually(df):
    print("Old Shape: ", df.shape)
    
    #Accuracy and Missrate
    for i in range(int(df.shape[1]/6)):
        col_acc_rate = df.columns[8+6*i:10+6*i]
        for j in col_acc_rate:
            df.drop(df[df[j] > 1].index, inplace=True)
    print("New Shape: ", df.shape)

In [ ]:
remove_outliers_manually(data)

A continuación, se va a proceder con la eliminación de los outliers, es decir, se va a prescindir de aquellos valores que se consideren anómalos. Para ello se van a emplear dos funciones *.describe()* y *sns.distplot(col)*. La primera  se utiliza para representar la distribución de los datos en un gráfico. Va a permitir visualizar la forma en que los datos están distribuidos en los datasets disponibles. A simple vista, se puede identificar como hay valores máximos que están muy alejados del percentil 75, por lo que se puede intuir que se trata de un outlier.

A su vez, la función `countplot` es útil para identificar cuantos valores positivos y negativos hay de dislexia. Se va a representar visualmente, permitiendo comparar si la muestra que se está usando es representativa de la población.

In [ ]:
data.describe()

Tras observar la distribución del *dataset* original y comprobar como sí presenta outliers, se va a proceder con el análisis y la eliminación de aquellos valores que no aporten información válida y vayan a afectar negativamente al modelo.

La función `distplot` se va a emplear para identificar la forma en que los datos están distribuidos, verificando si siguen una distribución normal o no, como afecta la presencia de outliers a la distribución y cual es la media y la desviación estándar. Es decir, se va a representar visualmente la distribución de los datos de cada variable.

La función `boxplot` representa el rango intercuartílico (IQR) de los datos. La caja (gráfico) va desde el primer cuartil (Q1) hasta el tercer cuartil (Q3), y su longitud representa el rango intercuartílico (IQR = Q3 - Q1). Los valores atípicos (outliers) son aquellos que están fuera del rango definido por 1,5 veces el IQR.

In [ ]:
def plot_dist(df):
    
    rest = df.shape[1] % 4
    rows = df.shape[1]//4
    
    if rest == 0:
        rows = rows
    else:
        rows = rows + 1
        
    fig, ax = plt.subplots(ncols = 4, nrows = rows, figsize = (15, 150))
    i = 0
    ax = ax.flatten()

    for col in df.columns:
        plt.title(col)
        sns.distplot(df[col], ax=ax[i])
        i+=1
    plt.tight_layout(pad = 0.5, w_pad = 0.7, h_pad = 2.0)

In [ ]:
def plot_box_whisker(df):
    
    rest = df.shape[1] % 4
    rows = df.shape[1]//4
    
    if rest == 0:
        rows = rows
    else:
        rows = rows + 1

    fig, ax = plt.subplots(ncols = 4, nrows = rows, figsize = (15, 150))
    i = 0
    ax = ax.flatten()
    
    for col in df.columns:
        plt.title(col)
        df.boxplot([col], ax=ax[i])
        i+=1
    plt.tight_layout(pad = 0.5, w_pad = 0.7, h_pad = 2.0)

Se va a trabajar solo con las variables relativas a las preguntas

In [ ]:
data_Q = data.drop(columns = ['Gender','Nativelang','Otherlang','Age','Dyslexia'], axis = 1)

In [ ]:
#plot_dist(data_Q)

### Percentile method

Para la eleminación de outliers existen varias técnicas, como Z-score method, Inter Quartile Range Method o Percentile method. la técnica que se ha elegido es el método de percentil ya que es fácil de entender y aplicar, y no requiere de una distribución específica. Como se ha podido observar, no todas las variables tienen la misma distribución, lo que podría suponer un problema utilizando Z-score o el método del Intercuartil.

Se trata de un método robusto a la presencia de valores atípicos en los datos, lo que lo hace menos propenso a ser afectado por ellos, y permite ajustar los límites al dataset con el que se esté trabajando. En este caso, para no prescindir de un gran número de datos, aun eliminando aquellos datos que influían negativamente al modelo, se han establecido como umbral superior el 0.999 e inferior el 0.001.

In [ ]:
def remove_outliers(ds):
    print("Old Shape: ", ds.shape)
    
    rng = ds.columns
    for col in rng:
        #first percentile
        min_threshold = ds[col].quantile(0.001)
        #third percentile
        max_threshold = ds[col].quantile(0.999)
        
        ds.drop(ds[(ds[col] < min_threshold) | (ds[col] > max_threshold)].index, inplace=True)
        
    print("New Shape: ", ds.shape)

In [ ]:
remove_outliers(data_Q)

Representando de nuevo los gráficos de distribución se puede observar como la presencia de outliers ha disminuido considerablemente:

In [ ]:
#plot_dist(data_Q)

### Selección de características

Se eliminan aquellas filas que contienen valores anómalos de los datasets inciales, recuperando las columnas: 'Gender', 'Nativelang', 'Otherlang', 'Age', 'Dyslexia'

In [ ]:
data = data.loc[data_Q.index]

Se van a definir dos funciones nuevas, `corr_hit_miss`y `corr_acc_missrate`, para comparar como de correladas están las variables *Clicks*, *Hits*, *Misses* y *Score* y la suma de las variables *Hits y Misses* con *Clicks*.

In [ ]:
def corr_hit_miss(ds_orig, nombre):
    ds = ds_orig.copy()
    print(nombre+" dataset")
    for i in range(32):
        try:        
            print("Question", str(i+1))
            #Pearson correlation coef
            correlation_misses = ds['Clicks'+str(i+1)].corr(ds['Misses'+str(i+1)])
            correlation_hits = ds['Clicks'+str(i+1)].corr(ds['Hits'+str(i+1)])
            correlation_score = ds['Clicks'+str(i+1)].corr(ds['Score'+str(i+1)])

            ds['Hits + Misses'+str(i+1)] = ds['Hits'+str(i+1)] + ds['Misses'+str(i+1)]
            correlation_sum = ds['Clicks'+str(i+1)].corr(ds['Hits + Misses'+str(i+1)])

            print("Misses", str(i+1), correlation_misses)
            print("Hits", str(i+1), correlation_hits)
            print("Score", str(i+1), correlation_score)
            print("Hits + Misses", str(i+1), correlation_sum)
        except KeyError:
            print("Question "+str(i+1)+" not in the dataset")
            
    print("\n")

In [ ]:
def corr_acc_missrate(ds_orig, nombre):
    ds = ds_orig.copy()
    print(nombre+" dataset")
    for i in  range(32):
        try:
            print("Question"+str(i+1))
            #Pearson correlation coeff
            correlation_acc = ds['Hits'+str(i+1)].corr(ds['Accuracy'+str(i+1)])
            correlation_missrate = ds['Misses'+str(i+1)].corr(ds['Missrate'+str(i+1)])
            correlation_accs_rate = ds['Accuracy'+str(i+1)].corr(ds['Missrate'+str(i+1)])

            print("Accuracy"+str(i+1),correlation_acc)
            print("Missrate"+str(i+1),correlation_missrate)
            print("Between Accuracy and Missrate"+str(i+1),correlation_accs_rate)
        except KeyError:
            print("Question "+str(i+1)+" not in the dataset")
            
    print("\n")

Se comprobará tanto para los dataset de entrenamiento, como de prueba y diferenciando entre los cuatro casos distintos tras aplicar las técnicas de gestión de outliers

In [ ]:
corr_hit_miss(data, "Model")
corr_acc_missrate(data, "Model")

In [ ]:
for i in range(32):
    columns = ['Accuracy'+str(i+1), 'Missrate'+str(i+1), 'Score'+str(i+1)]
    try:
        
        data = data.drop(columns, axis=1)
    except KeyError:
        print("Colmuns not found: ",columns)

In [ ]:
data.head()

### Divide data

Tras realizar la limpieza y preparar los datos que se van a emplear para diseñar el modelo, se va a proceder con la separación entre las variables que se van a empelar para entrenar el modelo y la variable que se quiere predecir. Es decir, se va a diferenciar entre **X**, set con todas las variables excepto si una persona es disléxica o no, e **y**, set que contiene los valores corresondientes a la variable *Dyslexia*.

In [ ]:
from sklearn.model_selection import train_test_split

y = data['Dyslexia']
X = data.loc[:, data.columns != 'Dyslexia']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4, stratify = y)

# Data Standardization -> Good practice working with KNN (based on the distance)
X_train_st = preprocessing.StandardScaler().fit(X_train).transform(X_train.astype(float))
X_test_st = preprocessing.StandardScaler().fit(X_test).transform(X_test.astype(float))

In [ ]:
# Data type
print("Tipo de datos de X_train_st:", type(X_train))
print("Tipo de datos de y_train:", type(y_train))

In [ ]:
#Dyslexia histogram 
dyslexia_train = y_train.value_counts()
dyslexia_test = y_test.value_counts()

fig, ax = plt.subplots(1, 2, figsize=(20, 5))

sns.barplot(x=dyslexia_train.index, y=dyslexia_train.values, ax=ax[0])
ax[0].set_title('Train set')
ax[0].set_ylim([0, 2000])
ax[0].grid(True)
ax[0].annotate('Non dyslexia values: ' + str(dyslexia_train[0]), xy=(1, dyslexia_train[0]), xytext=(-0.25, dyslexia_train[0] + 20))
ax[0].annotate('Dyslexia values: ' + str(dyslexia_train[1]), xy=(1, dyslexia_train[1]), xytext=(0.82, dyslexia_train[1] + 20))

sns.barplot(x=dyslexia_test.index, y=dyslexia_test.values, ax=ax[1])
ax[1].set_title('Test set')
ax[1].set_ylim([0, 600])
ax[1].grid(True)
ax[1].annotate('Non dyslexia values: ' + str(dyslexia_test[0]), xy=(1, dyslexia_test[0]), xytext=(-0.25, dyslexia_test[0] + 5))
ax[1].annotate('Dyslexia values: ' + str(dyslexia_test[1]), xy=(1, dyslexia_test[1]), xytext=(0.82, dyslexia_test[1] + 5))

plt.show()


## Modeling Training

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, jaccard_score

Se va a proceder a entrenar el modelo. Como se mencionó al inicio del documento se van a emplear cinco algoritmos distintos de Machine Learning, de los cuales se eligirá el que prediga los casos positivos y negativos.

Antes de comenzar con el entrenamiento del modelo se van a definir varias funciones con el objetivo de simplificar el código:

- `plot_confusion_matrix(cm, classes)`: toma como argumentos de entrada la matriz de confusión del modelo y los valores que se quieren evaluar, en este caso 0's y 1's. Mostrará por pantalla la representación de los aciertos y fallos comparando que casos se han predicho correctamente y cuales no.
- `knn_accuracy(X_train, y_train, X_test, y_test, Ks)`: uno de los algoritmos de Machine Learning que se va a entrenar es K-Nearest Neighbors, por lo que es necesario escoger cuidadosamente el número de clusters con el que se va a entrenar el modelo. Mediante esta función, pasando como argumentos de entrada los datos usados para el train y el test y el máximo de clusters que se quieren probar, se imprimirá por pantalla la exactitud (*accuracy*) que tiene el modelo en función de los distintos valores de k
- `k_fold_cv(X, y, n, model)`: K-fold  cross-validation se emplea como técnica de evaluación del modelo recursiva. Pasando como argumentos de entrada los datos, el modelo que se quiere testear y la cantidad de iteración que se van a realizar, la función va a dividir de diferente forma los datos n veces y calcular la media de los resultados obtenidos en cada iteración al final. Se ha especificado que devuelva la excatitud del modelo (*score*) y el F1-score de las dos clases, es decir, tanto de los 0's como de los 1's
- `evaluate_model(X_train, y_train, X_test, y_test, model)`: esta función va a devolver distintas evaluaciones del modelo, como por ejemplo *Recall*, *Accuracy*, *Jaccard index* o *F1-score*, así como la representación de la matriz de confusión del modelo que se especifique.

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes):
    # Create figure and axes
    fig, ax = plt.subplots()
    im = ax.imshow(cm, cmap='Blues')

    #add value count to each cell
    fmt = 'd'
    thresh = cm.max() / 3.
    
    #labels value count
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #set axis labels
    ax.set_xticks(np.arange(len(classes)))
    ax.set_yticks(np.arange(len(classes)))
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')

    #colorbar
    cbar = ax.figure.colorbar(im, ax=ax)

    plt.show()

In [ ]:
#calculate nº clusters KNN
def knn_accuracy(X_train, y_train, X_test, y_test, Ks):
    
    for n in range(1,Ks):
        model = KNeighborsClassifier(n_neighbors=n).fit(X_train, y_train)
        y_hat = model.predict(X_test)
        
        acc = metrics.accuracy_score(y_test, y_hat)
        print(f"Value of k: {n} - Accuracy: {acc:.2f}")

In [ ]:
def k_fold_cv(X, y, n, model):
    kf = KFold(n_splits = n)
    
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    
    #accuracy
    scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index].values, X.iloc[test_index].values
        y_train, y_test = y.iloc[train_index].values, y.iloc[test_index].values
        #model prediction 
        y_hat = model.predict(X_test)
        #evaluate the performance of a model (accuracy score)
        score = model.score(X_test,y_test)
        scores.append(score)
        
    
    return "Accuracy: "+str(np.mean(scores))

In [ ]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
    
    y_hat = model.predict(X_test)
    
    #different evaluations
    print("Train set Accuracy: ", metrics.accuracy_score(y_train, model.predict(X_train)))
    print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_hat))
    '''
    print("Recall 0's: ", recall_score(y_test, y_hat, pos_label=0))
    print("Recall 1's: ", recall_score(y_test, y_hat, pos_label=1))
    print("F1 score 0's: ", f1_score(y_test, y_hat, pos_label=0))
    print("F1 score 1's: ", f1_score(y_test, y_hat, pos_label=1))
    '''
    print("Jaccard index 0's: ", jaccard_score(y_test, y_hat, pos_label = 0))
    print("Jaccard index 1's: ", jaccard_score(y_test, y_hat, pos_label = 1))
    print("\n")
    
    #classification report
    print (classification_report(y_test, y_hat))
    print("\n")

    #compute confusion matrix
    cm = confusion_matrix(y_test, y_hat, labels=[0,1])
    np.set_printoptions(precision = 2)

    #plot confusion matrix
    plt.figure()
    plot_confusion_matrix(cm, classes=['No = 0','Yes = 1'])

## 1. K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_accuracy(X_train_st, y_train, X_test_st, y_test, 10)

In [ ]:
k = 2
#Train Model and Predict  
neighbor = KNeighborsClassifier(n_neighbors = k).fit(X_train_st,y_train)
y_hat_KNN = neighbor.predict(X_test_st)

In [ ]:
k_fold_cv(X, y, 10, neighbor)

In [ ]:
evaluate_model(X_train_st, y_train, X_test_st, y_test, neighbor)

## 2. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Train Model and Predict  
mod_LR = LogisticRegression(solver='liblinear').fit(X_train_st,y_train)
y_hat_LR = mod_LR.predict(X_test_st)

# Predict the probability of each class for a given input
y_hat_prob = mod_LR.predict_proba(X_test_st)

In [ ]:
k_fold_cv(X, y, 10, mod_LR)

In [ ]:
evaluate_model(X_train_st, y_train, X_test_st, y_test, mod_LR)

## 3. Support Vector Machines

In [ ]:
from sklearn import svm

In [ ]:
# Model: svm.SVC(kernel='rbf')
# kernel='rbf'" where 'rbf' stands for radial basis function

In [ ]:
# Train Model and Predict 
mod_SVM = svm.SVC(kernel='poly', degree=15, coef0=7, random_state=42).fit(X_train, y_train) 
y_hat_SVM = mod_SVM.predict(X_test)

In [ ]:
k_fold_cv(X, y, 10, mod_SVM)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, mod_SVM)

# 4. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Train Model and Predict 
mod_rfc = RandomForestClassifier(n_estimators = 30, max_depth = 7, min_samples_leaf = 7).fit(X_train , y_train)
y_hat_RF = mod_rfc.predict(X_test)

In [ ]:
k_fold_cv(X, y, 10, mod_rfc)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, mod_rfc)

## 5. Tree Decision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.tree as tree

In [ ]:
# Train Model and Predict 
tree = DecisionTreeClassifier(criterion = "entropy", max_depth = 15).fit(X_train, y_train)
y_hat_tree = tree.predict(X_test)

In [ ]:
k_fold_cv(X, y, 10, tree)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, tree)